# Analysis of Terrorism Incidents

### a project of 

# Center of Data and Text Engineering and Mining (CoDTeEM)


## Pakistan Data (Spreadsheet)

In [6]:
import numpy as np
import pandas as pd
import urllib
import time

from datetime import datetime, date

from IPython.display import display, HTML
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

import math
# library for geo-json
from shapely.geometry import shape, Point, mapping

import ast
import string
import json

pd.options.mode.chained_assignment = None
mapbox_access_token = 'pk.eyJ1IjoibWFrcy1zaCIsImEiOiJjajRnMGhyNzcxZGFzMnd1ZnZteHF1YXo3In0.5R_XD1wl8F7ffCjAN1yxLg'


In [7]:
terror_data = pd.read_csv('data/globalterrorismdb_0718dist.csv', encoding='ISO-8859-1',
                          usecols=[0, 1, 2, 3, 8, 11, 12, 13, 14, 29, 35, 84, 98,99])
terror_data = terror_data.rename(
    columns={'eventid':'id', 'iyear':'year', 'imonth':'month', 'iday':'day',
             'country_txt':'country', 'provstate':'state', 'city':'city', 'targtype1_txt':'target',
             'weaptype1_txt': 'weapon', 'attacktype1_txt': 'attacktype', 'nkill':'nkill', 
             'nwound':'nwound', 'addnotes': 'info'})

terror_data['nkill'] = terror_data['nkill'].fillna(0).astype(int)
terror_data['nwound'] = terror_data['nwound'].fillna(0).astype(int)
terror_data = terror_data.dropna(how='any', subset=['latitude', 'longitude'])
print('Matrix size:', terror_data.shape)

Matrix size: (169286, 14)


In [8]:
terror_pak = terror_data[terror_data['country'] == 'Pakistan']
terror_india = terror_data[terror_data['country'] == 'India']
terror_afghan = terror_data[terror_data['country'] == 'Afghanistan']

print('Pakistan size:', terror_pak.shape)
print('India size:', terror_india.shape)
print('Afghanistan size:', terror_afghan.shape)

Pakistan size: (13820, 14)
India size: (11111, 14)
Afghanistan size: (11559, 14)


In [9]:
terror_pak_assassination = terror_pak[terror_pak['attacktype'] == 'Assassination']
print('Matrix size:', terror_pak_assassination.shape)
#terror_pak_assassination.head(5)

Matrix size: (1351, 14)


In [11]:
terror_pak.loc[:, 'day'] = terror_pak.apply(lambda row: str(row['id'])[6:8] if row['day'] == 0 else row['day'], axis=1)
#terror_pak.loc[:, 'date'] = pd.to_datetime(terror_pak[['day', 'month', 'year']])

terror_pak = terror_pak.drop_duplicates(['latitude', 'longitude', 'nkill'])
terror_peryear = np.asarray(terror_pak.groupby('year').year.count())
terror_years = np.arange(1970, 2016)
terror_years = np.delete(terror_years, [1])        
        
terror_pak

,id,year,month,day,country,state,city,latitude,longitude,attacktype,target,weapsubtype1_txt,nkill,nwound
585,1.970110e+11,1970,11,1,Pakistan,Sindh,Karachi,24.891115,67.143311,Assassination,Government (Diplomatic),NaN,4,20
2217,1.974020e+11,1974,2,2,Pakistan,Sindh,Karachi,24.891115,67.143311,Bombing/Explosion,Maritime,Automatic or Semi-Automatic Rifle,0,0
2705,1.974120e+11,1974,12,9,Pakistan,North-West Frontier Province,Peshawar,34.006004,71.537430,Bombing/Explosion,Government (Diplomatic),Unknown Explosive Type,0,2
2744,1.975000e+11,1975,0,00,Pakistan,Punjab,Rawalpindi,33.594013,73.069077,Bombing/Explosion,Airports & Aircraft,Unknown Explosive Type,0,0
2795,1.975020e+11,1975,2,8,Pakistan,North-West Frontier Province,Peshawar,34.006004,71.537430,Assassination,Government (General),Handgun,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173547,2.017040e+11,2017,4,2,Pakistan,Federally Administered Tribal Areas,Kachkina,33.886634,70.010088,Bombing/Explosion,Private Citizens & Property,"Projectile (rockets, mortars, RPGs, etc.)",0,0
173568,2.017040e+11,2017,4,2,Pakistan,Federally Administered Tribal Areas,Shingak,33.842159,69.987799,Bombing/Explosion,Business,"Projectile (rockets, mortars, RPGs, etc.)",0,0
173617,2.017040e+11,2017,4,5,Pakistan,Balochistan,Shiktaak,28.584053,65.416645,Armed Assault,Business,Automatic or Semi-Automatic Rifle,4,0
173788,2.017040e+11,2017,4,9,Pakistan,Federally Administered Tribal Areas,Bara,33.916433,70.882702,Bombing/Explosion,Educational Institution,Unknown Explosive Type,0,0
